In [15]:
import eikon as ek
import pandas as pd
from fetching_files import *
ek.set_app_key("5f62315c837645a78ff033f35644b3ba3ef5753a")
from importlib import reload

In [16]:
# Reloading the 'open_and_clean' module and re-importing
import preprocess.file_preprocess.open_and_clean
reload(preprocess.file_preprocess.open_and_clean)
from preprocess.file_preprocess.open_and_clean import *

# Reloading the 'filter_out_relevant_data' module and re-importing
import preprocess.file_preprocess.filter_out_relevant_data
reload(preprocess.file_preprocess.filter_out_relevant_data)
from preprocess.file_preprocess.filter_out_relevant_data import *

# Reloading the 'preprocess_to_df' module and re-importing
import preprocess.file_preprocess.preprocess_to_df
reload(preprocess.file_preprocess.preprocess_to_df)
from preprocess.file_preprocess.preprocess_to_df import *

# Reloading the 'find_and_remove_duplicates' module and re-importing
import preprocess.file_preprocess.find_and_remove_duplicates
reload(preprocess.file_preprocess.find_and_remove_duplicates)
from preprocess.file_preprocess.find_and_remove_duplicates import *

# Reloading the 'getters_and_counts' module and re-importing
import metricsUtils.getters_and_counts
reload(metricsUtils.getters_and_counts)
from metricsUtils.getters_and_counts import *

import metricsUtils.calculate_new_metrics
reload(metricsUtils.calculate_new_metrics)
from metricsUtils.calculate_new_metrics import *

#Reloading the 'segment_pipeline' module and re-importing
import pipelines.segment_pipeline
reload(pipelines.segment_pipeline)
from pipelines.segment_pipeline import *

import preprocess.configurations.configurations
reload(preprocess.configurations.configurations)
from preprocess.configurations.configurations import * 

import firm 
reload(firm)
from firm import Firm

import metricsUtils.impuding_calculations
reload(metricsUtils.impuding_calculations)
from metricsUtils.impuding_calculations import *

In [25]:
def find_wacc(year): 
    df = pd.read_csv(rf"ren_alternative/ren_alternative_for_time_-{year}y.csv")
    roe = pd.read_csv(rf"alt_industry_roe\industry_roe_for_year_-{year}y.csv")
    columns_to_check = ['Revenue', 'Company Market Cap', 'Total Debt', 'Price Close', 'Common Shares - Outstanding - Total', 'Total Assets', "Interest Expense",  "Interest Bearing Liabilities - Total"]
    df_new = df.dropna(subset=columns_to_check)
    df_new = df_new[(df_new['Total Assets'] != 0) & (df_new["Revenue"] > 20000000)] 
    df_new2 = fill_nan_wacc_tax_rate(df_new)
    df_FEPS = df_new2[((~df_new2['Earnings Per Share - SmartEstimate®'].isna())&(df_new2['Earnings Per Share - SmartEstimate®'])!=0 ) & ((df_new2['Earnings Per Share - SmartEstimate®.1']!=0)&(~df_new2['Earnings Per Share - SmartEstimate®.1'].isna()))]
    roe_estimates = calculate_median_industry_roe(roe, "TRBC Activity Code")
    firm_objects = create_firm_objects(df_FEPS, roe_estimates, 'TRBC Activity Code')

    wacc = []
    coe = []
    cod = []
    firm_names = []
    non_converging_firms = []

    # Iterate through each firm object
    for firm_name, firm in firm_objects.items():
        coe_result = firm.find_cost_of_equity()

        if coe_result is None:
            non_converging_firms.append(firm_name)
            wacc_result = None  # Or set a default value
            coe_result = None  # Or set a default value
        else:
            wacc_result = firm.calculate_gls_cost_of_capital()

        wacc.append(wacc_result)
        coe.append(coe_result)
        cod.append(firm.calculate_cost_of_debt())
        firm_names.append(firm_name)

    # Create a DataFrame from the results
    wacc_df = pd.DataFrame({
        "Firm Name": firm_names, 
        "WACC": wacc, 
        "COE": coe, 
        "COD": cod
    })
    return wacc_df

for i in range(10): 
    wacc_df = find_wacc(i)
    print(wacc_df)
    wacc_df.to_csv(rf"ren_alternative\wacc_for_time_-{i}y.csv", index=False)

       Firm Name      WACC       COE       COD
0      000539.SZ       NaN       NaN  0.028785
1      000629.SZ  0.042838  0.056250  0.032144
2      002545.SZ  0.086633  0.126500  0.031242
3      071320.KS  0.017861  0.060703  0.017506
4        2083.SE  0.039839  0.086875  0.024499
..           ...       ...       ...       ...
103      TTPW.NS  0.044186  0.093500  0.078243
104      UN01.DE  0.007431  0.145000  0.034612
105  UNACEMC1.LM  0.054606  0.092000  0.047682
106      VERB.VI  0.029958  0.070930  0.017110
107       XEL.OQ  0.033765  0.082102  0.035983

[108 rows x 4 columns]
       Firm Name      WACC       COE       COD
0      000539.SZ       NaN       NaN  0.024083
1      002060.SZ  0.027859  0.105000  0.035595
2      002545.SZ  0.073690  0.102250  0.036766
3      071320.KS  0.017057  0.044508  0.017936
4        1811.HK  0.041284  0.046500  0.059742
..           ...       ...       ...       ...
101      TTPW.NS  0.037392  0.070750  0.071480
102      UN01.DE  0.008995  0.172500

In [7]:
roe

,Instrument,TRBC Activity Code,Return On Equity - Actual,"Return on Average Common Equity - %, TTM",year
0,BRKa.N,5440101010,6.290000,-4.663796,FY0
1,GE.N,5440101010,7.389000,1.578506,FY0
2,MMM.N,5440101010,38.380000,38.813491,FY0
3,TECK.N,5120108010,20.100000,16.869508,FY0
4,WMT.N,5430102010,21.500000,14.604929,FY0
...,...,...,...,...,...
5968,4189.T,5110101010,14.100000,14.130187,FY0
5969,300769.SZ,5110101010,40.090000,39.297756,FY0
5970,900290.KQ,5110101011,7.795826,7.795826,FY0
5971,7347.T,5510202010,9.847310,9.847310,FY0
